### 1. setting matplotliba and importing all libraries

In [ ]:
#importing libraries
import matplotlib
matplotlib.use("Agg")
#We need to set the matplotlib backend to Agg to indicate to create a non-interactive that will simply be saved to disk

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from dhruv.nn.conv import minivggnet
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import argparse
from tensorflow.keras.layers import BatchNormalization


### 2. writing path for output and load data

In [8]:
args = {}
args["output"] = "output"


print("[Info] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

[Info] loading CIFAR-10 data...


### 3. label dataset 

In [9]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


### 4. compiling Network

In [10]:

print("[Info] compiling network...")
opt = SGD(learning_rate=0.01, decay=0.01 / 40, momentum=0.9, nesterov=True)
model = minivggnet.MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy",optimizer=opt, metrics=["accuracy"])


[Info] compiling network...


/home/umair-ali/Desktop/phd deep learning after mid/codes to make operational and understand/tf_phd/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
/home/umair-ali/Desktop/phd deep learning after mid/codes to make operational and understand/tf_phd/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### 4. traingin network

In [11]:
epoch_size = 40

In [12]:
print("[Info] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=epoch_size, verbose=1)


[Info] training network...
Epoch 1/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.3760 - loss: 1.9810 - val_accuracy: 0.5692 - val_loss: 1.2586
Epoch 2/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5831 - loss: 1.2144 - val_accuracy: 0.6288 - val_loss: 1.0676
Epoch 3/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6436 - loss: 1.0259 - val_accuracy: 0.6668 - val_loss: 0.9911
Epoch 4/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6940 - loss: 0.8734 - val_accuracy: 0.7248 - val_loss: 0.7890
Epoch 5/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7216 - loss: 0.7958 - val_accuracy: 0.7436 - val_loss: 0.7466
Epoch 6/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7448 - loss: 0.7289 - val_accuracy: 0.7260 - val_loss: 0.7703
Epoch 7/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7589 - loss: 0.6798 - val_accuracy: 0.7682 - val_loss: 0.6749
Epoch 8/40
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7786 - loss

### 5. Evaluation

In [13]:
print("[Info] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))


[Info] evaluating network...
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
              precision    recall  f1-score   support

    airplane       0.85      0.84      0.84      1000
  automobile       0.92      0.90      0.91      1000
        bird       0.79      0.70      0.74      1000
         cat       0.72      0.62      0.67      1000
        deer       0.78      0.84      0.81      1000
         dog       0.71      0.79      0.75      1000
        frog       0.86      0.87      0.87      1000
       horse       0.84      0.89      0.87      1000
        ship       0.89      0.92      0.90      1000
       truck       0.88      0.89      0.89      1000

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.82     10000
weighted avg       0.83      0.83      0.82     10000



### 6.Plotting

In [14]:

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epoch_size), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, epoch_size), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, epoch_size), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, epoch_size), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on CIFAR-10")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])